In [3]:
from pandas import read_csv
from keplergl import KeplerGl

In [4]:
height = 400 # CAMBIAR!
city = "PE|lima"
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': -12.135520,
            'longitude': -77.021701,
            'zoom': 12.32053899007826
        }
    }
}

In [6]:
df = read_csv('../results/grid.csv')

## Deficit map

In [5]:
deficit_map = df[['hexagonID', 'orders', 'couriers', 'capacity']]

In [6]:
deficit_hexagons = deficit_map.loc[deficit_map['capacity'] < 0]

In [7]:
KeplerGl(height=height, data={'base': deficit_map, 'deficits': deficit_hexagons}, config=config).save_to_html(file_name='deficit_map.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to deficit_map.html!


## Fleet size map

In [7]:
df

,hexagonID,orders,couriers,capacity,demand_observations,neighbors,fleet_size,pool_benefit,is_hub,fleet_pool,required_fleet,is_receiver,is_provider,is_balanced
0,888e62c0e7fffff,4,2,-2,"[2, 1, 2, 4, 2, 4, 5, 3, 5, 2]","[""888e62c0e7fffff"", ""888e62c0e5fffff"", ""888e62...",Main.Models.FleetSize(Distributions.Poisson{Fl...,0.479570,True,"Dict{Symbol, Real}(:safety_fleet => 0)",4,True,False,False
1,888e62c707fffff,11,11,0,"[16, 6, 11, 9, 8, 10, 8, 9, 12, 15]",String[],Main.Models.FleetSize(Distributions.Poisson{Fl...,0.392836,False,"Dict{Symbol, Real}()",12,True,False,False
2,888e62c1d5fffff,11,6,-5,"[4, 14, 12, 12, 11, 7, 9, 17, 10, 10]","[""888e62c1d5fffff"", ""888e62c1c3fffff"", ""888e62...",Main.Models.FleetSize(Distributions.Poisson{Fl...,0.444723,True,"Dict{Symbol, Real}(:safety_fleet => 1)",14,True,False,False
3,888e62c095fffff,2,2,0,"[0, 1, 0, 2, 2, 1, 1, 0, 2, 0]",String[],Main.Models.FleetSize(Distributions.Poisson{Fl...,0.594148,False,"Dict{Symbol, Real}()",2,True,False,False
4,888e62c085fffff,1,2,1,"[1, 2, 3, 1, 0, 0, 4, 1, 1, 1]",String[],Main.Models.FleetSize(Distributions.Poisson{Fl...,0.546466,False,"Dict{Symbol, Real}()",1,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,888e62c589fffff,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",String[],Main.Models.FleetSize(Distributions.Poisson{Fl...,NaN,False,"Dict{Symbol, Real}()",-1,False,True,False
221,888e62ced7fffff,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",String[],Main.Models.FleetSize(Distributions.Poisson{Fl...,0.000000,False,"Dict{Symbol, Real}()",-1,False,True,False
222,888e62c18bfffff,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",String[],Main.Models.FleetSize(Distributions.Poisson{Fl...,0.000000,False,"Dict{Symbol, Real}()",-1,False,True,False
223,888e62c1c7fffff,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",String[],Main.Models.FleetSize(Distributions.Poisson{Fl...,0.371915,False,"Dict{Symbol, Real}()",-1,False,True,False


In [8]:
fleet_size = df[['hexagonID', 'required_fleet', 'capacity', 'demand_observations', 'fleet_size']]

In [18]:
# def required_fleet(row):
#     result = row['fleet_size'].split("), ")

#     return int(result[1].split(", :demand")[0])

In [19]:
# required = fleet_size.apply(required_fleet, axis=1)

In [20]:
# fleet_size.loc[:, 'required_fleet'] = required

In [9]:
KeplerGl(height=height, data={'base': fleet_size}, config=config).save_to_html(file_name='fleet_size_map.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to fleet_size_map.html!


## Fleet pool map

In [22]:
fleet_pool = df[['hexagonID', 'capacity', 'pool_benefit', 'is_hub', 'fleet_pool']]

In [23]:
def safety_fleet(row):
    result = row['fleet_pool']
    
    if result != 'Dict{Symbol, Real}()':
        return int(result.split("safety_fleet => ")[1][:-1])

    return 0

In [24]:
safety = fleet_pool.apply(safety_fleet, axis=1)

In [26]:
fleet_pool.loc[:, 'safety_fleet'] = safety

/var/folders/cm/z43qqh5n179fc7qhpyjm71p00000gn/T/ipykernel_12925/1488923038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fleet_pool.loc[:, 'safety_fleet'] = safety


In [27]:
hubs = fleet_pool.loc[fleet_pool['is_hub'] == 1]

In [28]:
KeplerGl(height=height, data={'base': fleet_pool, 'hubs': hubs}, config=config).save_to_html(file_name='fleet_pool_map.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to fleet_pool_map.html!


/Users/sebastiangrandaaltamirano/.pyenv/versions/3.10.4/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## Fleet allocation

In [30]:
relocations = read_csv('../results/relocations.csv')

In [31]:
KeplerGl(height=height, data={'base': df, 'relocations': relocations}, config=config).save_to_html(file_name='relocations_map.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to relocations_map.html!


/Users/sebastiangrandaaltamirano/.pyenv/versions/3.10.4/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## Benchmark

In [41]:
results = read_csv('../results/1.5_metrics.csv')

In [42]:
results[['time_improvement']].mean()

time_improvement    25.512674
dtype: float64

In [36]:
results

,saturation,avg_time_before,avg_time_after,time_improvement,unassigned_before,unassigned_after,unassigned_improvement
0,0.5,3.176367,2.633322,17.096404,90,90,0.000000
1,0.5,4.988344,3.555860,28.716632,94,94,0.000000
2,0.5,6.162925,4.457468,27.672858,89,89,0.000000
3,0.5,6.605439,4.623605,30.003062,85,85,0.000000
4,0.5,6.296557,5.037015,20.003652,97,97,0.000000
5,0.5,4.955131,3.676223,25.809765,103,103,0.000000
6,0.5,4.940649,4.000183,19.035263,101,101,0.000000
7,0.5,4.980294,3.469876,30.327889,106,107,-0.943396
8,0.5,7.193862,5.054947,29.732495,100,100,0.000000
9,0.5,4.659484,3.299012,29.197919,106,106,0.000000


In [382]:
relocated_couriers = read_csv('results.csv')

In [394]:
relocated_couriers['is_relocated'].describe()

count      3807
unique        2
top       False
freq       3318
Name: is_relocated, dtype: object